In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import json

# 베스트 셀레에서 책 url 크롤링

## start(driver, url)

아래와 같은 데이터를 리턴합니다.

다음은 베스트 셀러에 있는 책들의 상세페이지 url들을 담고있는 데이터 구조입니다.

```python
# 다음은 책들의 상세페이지의 url을 가지고 있는 book_urls의 예제입니다.
book_urls = [
    https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=40869703,
    https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=40869704,
    https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=40869705,
    https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=40869706,
    #추가 url...
]
```

In [3]:
# 페이지의 책들의 url을 크롤링
def crolling_page_url(driver):
    books = driver.find_elements(By.CSS_SELECTOR, '.bo3')
    
    page_urls = []
    for book in books:
        url = book.get_attribute('href')  # href 속성에서 링크 추출
        page_urls.append(url)
    return page_urls

def crolling_urls(driver):
   
    book_urls = []

    try:
        # 페이지 네비게이션 요소를 찾습니다.
        tmp = driver.find_element(By.CSS_SELECTOR, '.megaseller_rank')
        pages = tmp.find_elements(By.CSS_SELECTOR, '[class$="naviRank"]')

        
        for page_index in range(len(pages)):
            if(page_index == 10):
                next = driver.find_elements(By.CSS_SELECTOR, '.endse01 a')[1]
                ActionChains(driver).move_to_element(next).click().perform()
            try:
                # 페이지 네비게이션 버튼을 다시 찾아 클릭 (동적 로드 문제 해결)
                if(page_index != 0):
                    next_page = driver.find_elements(By.CSS_SELECTOR, '[class$="naviRank"]')[page_index]
                    next_page = next_page.find_element(By.CSS_SELECTOR, 'a')
                    ActionChains(driver).move_to_element(next_page).click().perform()

                # 현재 페이지의 책 URL 크롤링
                page_urls = crolling_page_url(driver)
                book_urls.extend(page_urls)

            except Exception as e:
                print(f"{page_index}페이지 클릭 에러: {e}")
                continue

    except Exception as e:
        print(f"페이지 네비게이션 크롤링 에러: {e}")
    print(f"크롤링된 책 URL 개수: {len(book_urls)}")
    return book_urls

def start_crolling_urls(driver, url):
    print("url 크롤링 시작")
    driver.get(url)
    time.sleep(1)  # 페이지가 로드될 시간을 확보

    book_urls = []
    
    categorys = driver.find_elements(By.CSS_SELECTOR, '.best_left_ul a')

    for category_index in range (len(categorys)):
        if(category_index == 0): 
            continue
        next_category = driver.find_elements(By.CSS_SELECTOR, '.best_left_ul > li > a')[category_index]
        ActionChains(driver).move_to_element(next_category).click().perform()
        book_urls.extend(crolling_urls(driver))

    return book_urls

# 책의 상세페이지 크롤링

## crolling_book(driver, url)

아래와 같은 데이터를 리턴합니다.

다음은 상세 페이지 크롤링으로 수집한 책 데이터의 구조입니다:

```python
# 책 정보와 리뷰 데이터를 담은 구조 예제
book_data = {
    "제목": "책 제목",
    "저자": "저자 이름",
    "출판사": "출판사 이름",
    "출판일": "YYYY-MM-DD",
    "정가": "가격",
    "리뷰": [
        {
            "작성자": "리뷰 작성자",
            "내용": "리뷰 내용",
            "별점": 5,  # 1부터 5까지의 숫자
            "작성 날짜": "YYYY-MM-DD"
        },
        {
            "작성자": "리뷰 작성자",
            "내용": "리뷰 내용",
            "별점": 4,
            "작성 날짜": "YYYY-MM-DD"
        }
        # 추가 리뷰 ...
    ]
}


In [4]:
# 출판사+출판일 분리하는 함수
def split_fixed_length(s, date_length=10):
    if len(s) < date_length:
        return s, ""  # 문자열이 고정된 길이보다 짧으면 그대로 반환
    res = []
    res.append(s[:-date_length])
    res.append(s[-date_length:] )

    return res

# 스크롤하면서 찾기
def scroll_down_to_end(driver):
    while True:
        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollBy(0, 2000);")
        time.sleep(0.2)  # 스크롤 후 로딩 대기
        
        try:
            target_element = driver.find_element(By.ID, "tabOrderer")
            print("태그를 찾았습니다")
            break  # 요소를 찾았으면 반복 종료
        except:
            pass
            #print("ID를 가진 요소를 찾지 못했습니다.")
        
        current_scroll_position = driver.execute_script("return window.pageYOffset;")
        page_height = driver.execute_script("return document.body.scrollHeight")
              
            
        if current_scroll_position >= page_height-2000:  # 더 이상 스크롤되지 않을 때 종료
            print("마지막 페이지까지 스크롤 완료")
            break

#리뷰 데이터 크롤링
def crolling_review_info(driver):
     # 리뷰 데이터 크롤링
    reviews = []
    
    cnt = driver.find_element(By.CSS_SELECTOR, "#wa_product_top1_wa_Top_Ranking_pnlRanking > div.info_list.Ere_fs15.Ere_ht18 > a:nth-child(5)") 
    # 리뷰가 없으면 return
    if cnt.text == '100자평(0)':
        print("리뷰가 없습니다.")
        return reviews
    
    try:
        scroll_down_to_end(driver);
        
        #time.sleep(1)
        while True:
            # "더보기" 버튼 클릭
            try:
                more_button = driver.find_element(By.ID, "divReviewPageMore")  # 버튼 ID
                more_button.find_element(By.TAG_NAME, "a").click()  # 버튼 클릭
                time.sleep(0.5)  # 페이지 로드 대기
            except Exception as e:
                print("더보기 버튼이 더 이상 존재하지 않습니다.")
                break
    
        # 모든 리뷰 항목 가져오기
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".review_list_wrap .hundred_list")  # 리뷰 전체를 감싸는 클래스
        for review_element in review_elements:
            try:
            
                # 별점 추출
                stars = review_element.find_elements(By.CSS_SELECTOR, ".HL_star img")
                rating = len([star for star in stars if "icon_star_on.png" in star.get_attribute("src")])  # 별점 개수 계산
                
                # 리뷰 내용 추출
                review_content_element = review_element.find_elements(By.CSS_SELECTOR, "[id^='spnPaper']")[1]
                content = review_content_element.text.strip()
                #content = ""
    
                # 작성자 추출
                author_element = review_element.find_element(By.CSS_SELECTOR, "a.Ere_sub_gray8")
                author = author_element.text.strip()
    
                # 작성 날짜 추출
                date_element = review_element.find_element(By.CSS_SELECTOR, "span.Ere_sub_gray8")
                date = date_element.text.strip()
    
                # 리뷰 데이터를 저장
                reviews.append({
                    "작성자": author,
                    "내용": content,
                    "별점": rating,
                    "작성 날짜": date
                })
            except Exception as e:
                print(f"리뷰 크롤링 중 오류 발생: {e}")
                continue

    except Exception as e:
        print(f"전체 리뷰 크롤링 중 오류 발생: {e}")
        
    return reviews;

#책정보 크롤링
def crolling_book_info(driver):
    # 책 정보 크롤링
    book_data = {}
    try:
        # 책정보 크롤링
        # 제목
        book_data["제목"] = driver.find_element(By.CSS_SELECTOR, ".Ere_bo_title").text
    
        # 저자, 출판사, 출판일 추출
        details_element = driver.find_element(By.CSS_SELECTOR, "li.Ere_sub2_title")
        detail_text = details_element.text  # 전체 텍스트 추출: "한강 (지은이) 창비 2014-05-19"
    
        # 출판사와 출판일 처리
        split_text = detail_text.replace("(지은이)", "").split()  # "(지은이)" 제거 후 분리
        book_data["저자"] = split_text[0];
        tmp = split_fixed_length(split_text[1])
        #print(tmp)
        
        book_data["출판사"] = tmp[0]
        book_data["출판일"] = tmp[1]

        # 책 분류 저장
        book_data["분류"] = driver.find_element(By.CSS_SELECTOR, "#ulCategory > li:nth-child(1) > a:nth-child(3)").text
        
        # 정가 (가격 정보)
        price_elements = driver.find_elements(By.CSS_SELECTOR, "div.info_list ul li div")
        for i in range(len(price_elements) - 1):
            if "정가" in price_elements[i].text:
                book_data["정가"] = price_elements[i + 1].text  # 정가 다음 div에서 가격 정보 추출
                break
    
    except Exception as e:
        print(f"책 정보 크롤링 에러: {e}")

    return book_data

# 책의 정보와 리뷰를 리턴
def crolling_book(driver, url):
    driver.get(url)
    #time.sleep(1)  # 페이지 로드 대기    

    book_data = crolling_book_info(driver);
    reviews = crolling_review_info(driver);
    book_data["리뷰"] = reviews
    #print(book_data)

    return book_data

#테스트
"""
# Selenium WebDriver 설정
# WebDriver 옵션을 설정합니다.
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저 창을 열지 않음 (백그라운드 실행)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
crolling_book(driver, "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=353017075&start=wz&ptid=9&utm_source=aladin&utm_medium=wizard&utm_campaign=choice&utm_content=welcome")
"""

'\n# Selenium WebDriver 설정\n# WebDriver 옵션을 설정합니다.\noptions = webdriver.ChromeOptions()\n# options.add_argument(\'--headless\')  # 브라우저 창을 열지 않음 (백그라운드 실행)\noptions.add_argument(\'--no-sandbox\')\noptions.add_argument(\'--disable-dev-shm-usage\')\ndriver = webdriver.Chrome(options=options)\ncrolling_book(driver, "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=353017075&start=wz&ptid=9&utm_source=aladin&utm_medium=wizard&utm_campaign=choice&utm_content=welcome")\n'

In [11]:
def add_book_info(book_collection, new_book):
    book_title = new_book["제목"]
    
    # 중복 확인
    if book_title in book_collection:
        print(f"'{book_title}' 책 정보가 이미 존재합니다.")
    else:
        book_collection[book_title] = new_book
        print(f"'{book_title}' 책 정보가 추가되었습니다. 총 {len(book_collection)}개")


if __name__ == "__main__":
    # Selenium WebDriver 설정
    # WebDriver 옵션을 설정합니다.
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창을 열지 않음 (백그라운드 실행)
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    #book_urls = [];
    #book_urls = start_crolling_urls(driver, "https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1&BestType=Bestseller")
    # JSON 파일 경로 지정
    book_urls=[]
    json_file_path = './book_urls2.json'
    with open(json_file_path, 'r', encoding='utf-8') as file:
        book_urls = json.load(file)
    
    #book_list = {};
    for url in book_urls:
        book = crolling_book(driver, url)
        if book:  # book이 비어있지 않으면
            add_book_info(book_list, book)

    print(f"총 {len(book_list)}개가 크롤링 되었습니다.")
    with open('book_list.json', 'w', encoding='utf-8') as file:
        json.dump(book_list, file, ensure_ascii=False, indent=4)

태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'리버 2' 책 정보가 추가되었습니다. 총 476개
리뷰가 없습니다.
'빨간내복야코 어린이 상식 : 이거 모르면 지구인 아님 1' 책 정보가 추가되었습니다. 총 477개
리뷰가 없습니다.
'도망가자, 바다면 더 좋고' 책 정보가 추가되었습니다. 총 478개
리뷰가 없습니다.
'2025 유휘운 행정법총론 요플 키워드 해설집' 책 정보가 추가되었습니다. 총 479개
리뷰가 없습니다.
'맥킨지의 전략적 프레임' 책 정보가 추가되었습니다. 총 480개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'망각 배터리 15' 책 정보가 추가되었습니다. 총 481개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'해독 혁명' 책 정보가 추가되었습니다. 총 482개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'생각에 관한 생각' 책 정보가 추가되었습니다. 총 483개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'행복의 기원' 책 정보가 추가되었습니다. 총 484개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'군주론 (무삭제 완역본)' 책 정보가 추가되었습니다. 총 485개
리뷰가 없습니다.
'2025 이기적 컴퓨터활용능력 1급 필기 + 실기 올인원' 책 정보가 추가되었습니다. 총 486개
리뷰가 없습니다.
'스모크 & 피클스' 책 정보가 추가되었습니다. 총 487개
리뷰가 없습니다.
'초예측 트럼프 2.0 새로운 시대' 책 정보가 추가되었습니다. 총 488개
태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
리뷰 크롤링 중 오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit

In [11]:
def add_book_info(book_collection, new_book):
    book_title = new_book["제목"]
    
    # 중복 확인
    if book_title in book_collection:
        print(f"'{book_title}' 책 정보가 이미 존재합니다.")
    else:
        book_collection[book_title] = new_book
        print(f"'{book_title}' 책 정보가 추가되었습니다. 총 {len(book_collection)}개")

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 열지 않음 (백그라운드 실행)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
book_list={}
with open("./book_list.json", 'r', encoding='utf-8') as file:
        book_list = json.load(file)

book = crolling_book(driver, "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=320694089")
add_book_info(book_list, book)
with open('book_list.json', 'w', encoding='utf-8') as file:
    json.dump(book_list, file, ensure_ascii=False, indent=4)

태그를 찾았습니다
더보기 버튼이 더 이상 존재하지 않습니다.
'차가운 자본주의' 책 정보가 추가되었습니다. 총 992개


In [9]:
print(len(book_list))

991


In [ ]:
print(book)